In [145]:
##pip install sodapy
from sodapy import Socrata
import pandas as pd
import datetime

In [146]:
##we filter all rows where vehicle_type column equals 'Car Service'
##client.timeout = 1000000 -> the connection doesnt time out after default time (10 seconds)
##limit by 1000000
client = Socrata("data.cityofnewyork.us", None)
client.timeout = 1000000
query = "SELECT * WHERE vehicle_type = 'Car Service' LIMIT 1000000 "

In [147]:
results = client.get("erm2-nwe9", query=query)

In [148]:
df = pd.DataFrame.from_records(results)

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8547 entries, 0 to 8546
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   unique_key                      8547 non-null   object
 1   created_date                    8547 non-null   object
 2   closed_date                     7334 non-null   object
 3   agency                          8547 non-null   object
 4   agency_name                     8547 non-null   object
 5   complaint_type                  8547 non-null   object
 6   descriptor                      8547 non-null   object
 7   location_type                   7201 non-null   object
 8   intersection_street_1           4873 non-null   object
 9   intersection_street_2           4871 non-null   object
 10  facility_type                   6768 non-null   object
 11  status                          8547 non-null   object
 12  due_date                        6241 non-null   

In [150]:
##choose relevant columns
df = df[['unique_key', 'created_date', 'agency', 'complaint_type', 'descriptor', 'incident_zip', 
'incident_address', 'street_name', 'city', 'borough', 'taxi_pick_up_location',
'latitude', 'longitude', 'location']]

In [151]:
##dropping null values from these columns as I'll change data type to number
df.dropna(subset=['latitude','longitude'], inplace=True)

##incident_zip has "N/A" value it needs to be classified as NaN first, then it can be dropped.
df['incident_zip'] = pd.to_numeric(df['incident_zip'], errors='coerce')
df.dropna(subset=['incident_zip'], inplace=True)

In [152]:
##change data types
df['unique_key'] = df['unique_key'].astype('int')
df['latitude'] = df['latitude'].astype('float')
df['longitude'] = df['longitude'].astype('float')
df['incident_zip'] = df['incident_zip'].astype('int')

#convert date column to datetime 
df['created_date'] = pd.to_datetime(df['created_date'], 
                                    format='%Y-%m-%dT%H:%M:%S.%f')
df.dtypes

unique_key                        int64
created_date             datetime64[ns]
agency                           object
complaint_type                   object
descriptor                       object
incident_zip                      int64
incident_address                 object
street_name                      object
city                             object
borough                          object
taxi_pick_up_location            object
latitude                        float64
longitude                       float64
location                         object
dtype: object

In [153]:
##creating year, mont, dayofweek columns
df['year'] = df['created_date'].dt.year
df['month'] = df['created_date'].dt.month
df['day'] = df['created_date'].dt.dayofweek

In [154]:
##filtering from 2017 to 2021, and creating a new df called "df311"
startdate=2017
enddate=2021
df311 = df[(df['year']>= startdate) & (df['year']< enddate)]
df311.reset_index(drop=True, inplace=True)

In [155]:
##confirm the data includes values from 2017 to 2020
print(min(df311['created_date']))
print(max(df311['created_date']))

2017-01-04 15:27:21
2020-12-31 09:28:13


In [156]:
df311.head()

,unique_key,created_date,agency,complaint_type,descriptor,incident_zip,incident_address,street_name,city,borough,taxi_pick_up_location,latitude,longitude,location,year,month,day
0,45906206,2020-03-27 11:33:26,TLC,For Hire Vehicle Complaint,Car Service Company Complaint,11203,462 EAST 52 STREET,EAST 52 STREET,BROOKLYN,BROOKLYN,"462 EAST 52 STREET, BROOKLYN, NY, 11203",40.649355,-73.928182,"{'latitude': '40.64935480219636', 'longitude':...",2020,3,4
1,45937007,2020-04-02 16:06:02,TLC,For Hire Vehicle Complaint,Car Service Company Complaint,10467,3211 PARKSIDE PLACE,PARKSIDE PLACE,BRONX,BRONX,"3211 PARKSIDE PLACE, BRONX, NY, 10467",40.874213,-73.874224,"{'latitude': '40.874212779486264', 'longitude'...",2020,4,3
2,45975433,2020-04-09 12:21:01,TLC,For Hire Vehicle Complaint,Car Service Company Complaint,10005,27 WILLIAM STREET,WILLIAM STREET,NEW YORK,MANHATTAN,"27 WILLIAM STREET, MANHATTAN (NEW YORK), NY, 1...",40.705565,-74.009940,"{'latitude': '40.705565064125295', 'longitude'...",2020,4,3
3,45995158,2020-04-13 23:02:59,TLC,For Hire Vehicle Complaint,Car Service Company Complaint,11203,KINGS COUNTY HOSPITAL,KINGS COUNTY HOSPITAL,BROOKLYN,BROOKLYN,"KINGS COUNTY HOSPITAL CENTER, BROOKLYN ,NY, 11203",40.656492,-73.944886,"{'latitude': '40.65649222055972', 'longitude':...",2020,4,0
4,46018036,2020-01-30 02:15:07,TLC,For Hire Vehicle Report,Driver Report - Passenger,10456,1070 WASHINGTON AVENUE,WASHINGTON AVENUE,BRONX,BRONX,"1070 WASHINGTON AVENUE, BRONX, NY, 10456",40.827821,-73.908816,"{'latitude': '40.8278213552831', 'longitude': ...",2020,1,3


In [157]:
##see null values in our final 311 data
df311.isnull().sum()

unique_key                 0
created_date               0
agency                     0
complaint_type             0
descriptor                 0
incident_zip               0
incident_address          34
street_name               34
city                     138
borough                    0
taxi_pick_up_location     54
latitude                   0
longitude                  0
location                   0
year                       0
month                      0
day                        0
dtype: int64

In [158]:
##export the data into a csv file
df311.to_csv ('311_Service_Request_TLC.csv', index = False, header=True)